In [124]:
import pandas as pd
import numpy as np
from time import perf_counter, perf_counter_ns
from time import sleep
from datetime import datetime as dt
from datetime import timedelta
import datetime
import json
import os
from sys import getsizeof
# import datetime
import random
# from py_vollib.black_scholes.implied_volatility import implied_volatility as bs_iv
# from py_vollib.black_scholes.greeks import analytical as greeks
# import time
# import ks_api_client
from kiteconnect import KiteConnect
# from kiteconnect import KiteTicker

In [121]:
# Prepare Exchange Books
current_datetime = dt(2021,5,17,9,30,0)
underlying_name =  'NIFTY'
current_datestring = current_datetime.strftime("%Y-%m-%d")

parent = os.path.dirname(os.getcwd())
historical_data_folder_name = 'historical data'
historical_data_folder_path = os.path.join(parent,historical_data_folder_name)


fno_data_folder_path = os.path.join(historical_data_folder_path,"FNO")
fno_file_paths = [os.path.join(fno_data_folder_path,f) \
    for f in os.listdir(fno_data_folder_path) \
        if os.path.isfile(os.path.join(fno_data_folder_path,f)) \
            & (f.split(".")[0].split("_")[0]==current_datestring)\
                ]

equity_data_folder_path = os.path.join(historical_data_folder_path,"Equity")
equity_file_paths = [os.path.join(equity_data_folder_path,f) \
    for f in os.listdir(equity_data_folder_path) \
        if os.path.isfile(os.path.join(equity_data_folder_path,f)) \
            & (f.split(".")[0].split("_")[0]==current_datestring)\
                ]

data_file_paths = fno_file_paths
data_file_paths.extend(equity_file_paths)

print(data_file_paths)
df = pd.concat(map(pd.read_csv,data_file_paths))
print(f'1) {getsizeof(df)/1_000_000}')

df['expiry_datetime'] = pd.to_datetime(df['expiry_date']) + \
                        timedelta (hours=15, minutes =30)

print(f'2) {getsizeof(df)/1_000_000}')
df['timestamp'] = pd.to_datetime(df['timestamp'])

print(f'3) {getsizeof(df)/1_000_000}')

df.rename(columns={'Ticker':'ticker',
                'LTP':'ltp',
                'BuyPrice':'buy_price',
                'SellPrice':'sell_price'},
                inplace=True)

print(f'4) {getsizeof(df)/1_000_000}')

df = df[['ticker','ltp','buy_price','sell_price','underlying','strike','call_put','expiry_datetime','timestamp']]

print(f'4.5) {getsizeof(df)/1_000_000}')

df = df[(df['underlying']==underlying_name) | (df['underlying'].isnull())]

print(f'5) {getsizeof(df)/1_000_000}')

df_group = df.groupby(['ticker','timestamp'])
df['duplicate_serial'] = df_group[['ltp']].cumcount()

print(f'7) {getsizeof(df)/1_000_000}')

df['duplicate_count'] = df_group['ltp'].transform('size')

print(f'8) {getsizeof(df)/1_000_000}')

del df_group
df['duplicate_fraction'] = pd.to_timedelta(df['duplicate_serial'] / df['duplicate_count'], unit='s')

print(f'9) {getsizeof(df)/1_000_000}')
df['timestamp'] = df['timestamp'] + df['duplicate_fraction']

print(f'10) {getsizeof(df)/1_000_000}')

df = df[['ticker','ltp','buy_price','sell_price','underlying','strike','call_put','expiry_datetime','timestamp']]
print(f'11) {getsizeof(df)/1_000_000}')
df

a_df = df[['ticker','underlying','strike','call_put','expiry_datetime']]
a_df.drop_duplicates(inplace=True)
print(f'12) {getsizeof(a_df)/1_000_000}')

['/Users/hg/Space/Code/AlgoTrader/historical data/FNO/2021-05-17.csv', '/Users/hg/Space/Code/AlgoTrader/historical data/Equity/2021-05-17.csv']
1) 2622.718639
2) 2660.429279
3) 2339.888839
4) 2339.888839
4.5) 1209.744879
5) 1209.744879
7) 1247.455519
8) 1285.166159
9) 1322.876799
10) 1322.876799
11) 1209.744879
12) 0.119201


/var/folders/rw/d270qq7d70j5rl2p2xpt2yfr0000gn/T/ipykernel_42249/3087327181.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a_df.drop_duplicates(inplace=True)


In [125]:
#Create Kite object
def generate_access_token(broker_secret, request_token) -> str:

        kite = KiteConnect(api_key=broker_secret['kite_api_key'])

        data = kite.generate_session(request_token=request_token, \
            api_secret=broker_secret['kite_api_secret'])
        print("Token Generated")
        del kite 

        return data['access_token']

parent = os.path.dirname(os.getcwd())
file_path = os.path.join(parent,'broker_secret.json')

with open (file_path, "r") as openfile:
    broker_secret = json.load(openfile)

is_kite_access_token_available = False
kite_request_token='aI5JfQQTyttmqmceF4l3nlXz7fIxrz1V'

if not is_kite_access_token_available:
        broker_secret['kite_access_token'] = \
            generate_access_token(broker_secret,kite_request_token)

        with open (file_path, "w") as outfile:
            json.dump(broker_secret,outfile)

kite = KiteConnect(api_key=broker_secret['kite_api_key'])
kite.set_access_token(broker_secret['kite_access_token'])

kite.positions()



Token Generated


{'net': [], 'day': []}

In [126]:
nfo = pd.DataFrame(kite.instruments("NFO"))
nfo
    

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
0,14038786,54839,NIFTY22FEB15100CE,NIFTY,0.0,2022-02-24,15100.0,0.05,50,CE,NFO-OPT,NFO
1,14039042,54840,NIFTY22FEB15100PE,NIFTY,0.0,2022-02-24,15100.0,0.05,50,PE,NFO-OPT,NFO
2,14039298,54841,NIFTY22FEB15150CE,NIFTY,0.0,2022-02-24,15150.0,0.05,50,CE,NFO-OPT,NFO
3,14039554,54842,NIFTY22FEB15150PE,NIFTY,0.0,2022-02-24,15150.0,0.05,50,PE,NFO-OPT,NFO
4,14039810,54843,NIFTY22FEB15200CE,NIFTY,0.0,2022-02-24,15200.0,0.05,50,CE,NFO-OPT,NFO
...,...,...,...,...,...,...,...,...,...,...,...,...
69813,13338626,52104,WIPRO22FEBFUT,WIPRO,0.0,2022-02-24,0.0,0.05,800,FUT,NFO-FUT,NFO
69814,14887938,58156,WIPRO22MARFUT,WIPRO,0.0,2022-03-31,0.0,0.05,800,FUT,NFO-FUT,NFO
69815,17080322,66720,ZEEL22APRFUT,ZEEL,0.0,2022-04-28,0.0,0.05,3000,FUT,NFO-FUT,NFO
69816,13339906,52109,ZEEL22FEBFUT,ZEEL,0.0,2022-02-24,0.0,0.05,3000,FUT,NFO-FUT,NFO


In [130]:
a = ['NFO:NIFTY22FEB17200CE','NFO:NIFTY22FEB17200PE','NFO:NIFTY22FEB17300CE','NFO:NIFTY22FEB17300PE']
kite.ltp(a)

{'NFO:NIFTY22FEB17200CE': {'instrument_token': 13420290, 'last_price': 476.95},
 'NFO:NIFTY22FEB17200PE': {'instrument_token': 13420546, 'last_price': 311.85},
 'NFO:NIFTY22FEB17300CE': {'instrument_token': 13421314, 'last_price': 421.1},
 'NFO:NIFTY22FEB17300PE': {'instrument_token': 13421570, 'last_price': 356.15}}

In [164]:
#LTP
# a = 'NIFTY20MAY2114750CE.NFO'
a = ['NIFTY20MAY2114750CE.NFO','NIFTY20MAY2114750PE.NFO','NIFTY20MAY2114650CE.NFO']

current_datetime = dt(2021,5,17,14,4,32)
initiation_time = perf_counter()
# sleep(2)
slippage = perf_counter() - initiation_time
current_datetime_slippage_adj = current_datetime + timedelta(seconds=slippage)
current_datetime_slippage_adj

if type(a) == list:
    ltp_df = pd.DataFrame(a)
    ltp_df = ltp_df.merge(df,left_on=0, right_on='ticker')
    ltp_df = ltp_df[ltp_df['timestamp']<=current_datetime_slippage_adj].drop_duplicates(subset='ticker',keep='last')
    ltp_df = ltp_df[['ticker','ltp']]
    return_value = ltp_df
else:
    k = df[(df['ticker']==a)&(df['timestamp']<=current_datetime_slippage_adj)][['ltp']].iloc[-1][0]
    return_value = k
return_value

,ticker,ltp
17388,NIFTY20MAY2114750CE.NFO,191.2
39903,NIFTY20MAY2114750PE.NFO,48.7
62171,NIFTY20MAY2114650CE.NFO,269.4


In [173]:
#Quote
a = 'NIFTY20MAY2114750CE.NFO'


current_datetime = dt(2021,5,17,14,4,32)
initiation_time = perf_counter()
# sleep(2)
slippage = perf_counter() - initiation_time
current_datetime_slippage_adj = current_datetime + timedelta(seconds=slippage)
current_datetime_slippage_adj

k = df[(df['ticker']==a)&(df['timestamp']<=current_datetime_slippage_adj)][['buy_price','sell_price']].iloc[-1]
return_value = k
return_value


buy_price     191.3
sell_price    191.7
Name: 17388, dtype: float64

In [174]:
type(return_value['buy_price'])

numpy.float64

In [128]:
kite.quote('NFO:NIFTY22FEB17200CE')

{'NFO:NIFTY22FEB17200CE': {'instrument_token': 13420290,
  'timestamp': datetime.datetime(2022, 1, 28, 12, 43, 7),
  'last_trade_time': datetime.datetime(2022, 1, 28, 12, 42, 53),
  'last_price': 480,
  'last_quantity': 100,
  'buy_quantity': 41750,
  'sell_quantity': 38950,
  'volume': 1001250,
  'average_price': 456.55,
  'oi': 463000,
  'oi_day_high': 551700,
  'oi_day_low': 463000,
  'net_change': 0,
  'lower_circuit_limit': 0.05,
  'upper_circuit_limit': 857,
  'ohlc': {'open': 424.75, 'high': 495.75, 'low': 408.25, 'close': 372.45},
  'depth': {'buy': [{'price': 478.9, 'quantity': 100, 'orders': 1},
    {'price': 478.55, 'quantity': 200, 'orders': 1},
    {'price': 478.5, 'quantity': 200, 'orders': 2},
    {'price': 478.3, 'quantity': 50, 'orders': 1},
    {'price': 478.25, 'quantity': 200, 'orders': 1}],
   'sell': [{'price': 480.1, 'quantity': 100, 'orders': 1},
    {'price': 480.15, 'quantity': 100, 'orders': 2},
    {'price': 480.3, 'quantity': 100, 'orders': 1},
    {'price'

'2021-05-17'

In [11]:
#Slice data
all_df = pd.read_csv("data_df/Data 2022-01-21 09_13_09 team.csv")
all_df['current_datetime'] = pd.to_datetime(all_df['current_datetime'])
all_df['expiry_datetime'] = pd.to_datetime(all_df['expiry_datetime'])
# all_df.rename(columns={'candle_5_0_close_last':'candle_5_0_close',
#                         'candle_5_0_open_last':'candle_5_0_open',
#                         'candle_5_5_close_last':'candle_5_5_close',
#                         'candle_5_5_open_last':'candle_5_0_open'},
#                         inplace=True)

from_time = dt(2022,1,21,10,1,0)
till_time = dt(2022,1,21,15,50,1)

slice = all_df[(all_df['current_datetime']>=from_time) &\
   (all_df['current_datetime']<=till_time)].head(3)

slice

,brokerage_pnl,candle_t_1_close,candle_t_1_open,candle_t_2_close,candle_t_2_open,current_datetime,current_ltp,current_pnl,current_position,expiry_datetime,is_closed,is_hedged,max_pnl,position_entry_ltp,straddle_strike,strangle_strike_low,strategy_pnl,trailing_pnl,underlying_name
3181,-0.02,17673.1,17661.9,17661.25,17622.1,2022-01-21 10:01:00.825733,17669.40,-900.02,NaN,2022-01-27 15:30:00,0.0,1.0,49.98,NaN,NaN,NaN,-900.0,-950.0,NIFTY
3182,-0.02,17673.1,17661.9,17661.25,17622.1,2022-01-21 10:01:01.792314,17669.40,-925.02,NaN,2022-01-27 15:30:00,0.0,1.0,49.98,NaN,NaN,NaN,-925.0,-975.0,NIFTY
3183,-0.02,17673.1,17661.9,17661.25,17622.1,2022-01-21 10:01:02.741716,17669.25,-925.02,NaN,2022-01-27 15:30:00,0.0,1.0,49.98,NaN,NaN,NaN,-925.0,-975.0,NIFTY


In [1]:
#Data Streaming using Websocket
import pandas as pd
import numpy as np
from time import perf_counter, perf_counter_ns
from time import sleep
from datetime import datetime as dt
from datetime import timedelta
import datetime
# import datetime
import random
from py_vollib.black_scholes.implied_volatility import implied_volatility as bs_iv
from py_vollib.black_scholes.greeks import analytical as greeks
import time
import ks_api_client
from kiteconnect import KiteConnect
from kiteconnect import KiteTicker

tokens = []
def set_kite (kite_api_key=None, kite_access_token=None):

    kite = KiteConnect(api_key=kite_api_key)
    kite.set_access_token(kite_access_token)
    kws = KiteTicker(kite_api_key,kite_access_token)
    return kite, kws


def get_instruments_book (kite):
    nse = pd.DataFrame(kite.instruments("NSE"))
    nfo = pd.DataFrame(kite.instruments("NFO"))
    kite_instruments_book = nfo.append(nse)
    kite_instruments_book['expiry'] =  pd.to_datetime(kite_instruments_book['expiry'], 
                                            format='%Y-%m-%d') 
    kite_instruments_book['expiry_datetime'] = kite_instruments_book['expiry'] + \
                                        timedelta (hours=15, minutes =30)
    return kite_instruments_book


def tokenLookup(instrument_df,symbol_list):
    """Looks up instrument token for a given script from instrument dump"""
    token_list = []
    for symbol in symbol_list:
        token_list.append(int(instrument_df[instrument_df.tradingsymbol==symbol].instrument_token.values[0]))
    return token_list


def get_fno_instrument_id (instruments_df,strike, underlying, call_put, expiry_datetime) -> str:
        
            
    instrument_id = str(instruments_df[(instruments_df['name']==underlying)
            &(instruments_df['instrument_type']==call_put)
            &(instruments_df['expiry_datetime']==expiry_datetime)
            &(instruments_df['strike']==strike)]['instrument_token'].iloc[0])
    
    if not isinstance(instrument_id,str):
        instrument_id = str(instrument_id.iloc[0])

    return instrument_id


def get_multiple_fno_instrument_id (instruments_df, fno_df) -> pd.Series:
        # fno_df should have underlying, call_put, expiry_datetime and strike column

    instrument_id = fno_df.merge(instruments_df,how='left',\
            left_on=['underlying','call_put','expiry_datetime','strike'], 
            right_on=['name','instrument_type','expiry_datetime','strike'])\
            ['instrument_token'].astype(int)

    return instrument_id

def on_close(ws, code, reason):
    print("closed connection on close: {} {}".format(code, reason))


def on_error(ws, code, reason):
    print("closed connection on error: {} {}".format(code, reason))


def on_noreconnect(ws):
    print("Reconnecting the websocket failed")


def on_reconnect(ws, attempt_count):
    print("Reconnecting the websocket: {}".format(attempt_count))


def on_order_update(ws, data):
    print("order update: ", data)


def on_connect(ws,response):
    global tokens
    print("on connect: {}".format(response))
    # Callback on successful connect.
    # Subscribe to a list of instrument_tokens (RELIANCE and ACC here).
    #print("on connect: {}".format(response))
    ws.subscribe(tokens)
    ws.set_mode(ws.MODE_FULL,tokens) # Set all token tick in `full` mode.
    #ws.set_mode(ws.MODE_FULL,[tokens[0]])  # Set one token tick in `full` mode.

counter = 0
combined_df = pd.DataFrame()
def on_ticks(ws,ticks):
    # Callback to receive ticks.
    # print("Ticks: {}".format(ticks))
    # print(ticks)
    # populate_df.delay(ticks)
    global counter
    global combined_df
    counter +=1
    tick_df = pd.DataFrame(ticks)[['instrument_token',\
        'last_price','volume','buy_quantity',\
        'sell_quantity','oi','timestamp','depth']]
    combined_df = combined_df.append(tick_df)
    # if dt.now().second == 0:
    #     combined_df.to_csv("tick_stream.csv",sep="|")
    #     print(f"saved,{counter}",end='\r')
    # else:
    print(f"         {counter}",end='\r')

def populate_df (ticks):
    global counter
    global combined_df
    counter +=1
    tick_df = pd.DataFrame(ticks)[['instrument_token',\
        'last_price','volume','buy_quantity',\
        'sell_quantity','oi','timestamp','depth']]
    combined_df = combined_df.append(tick_df)
    # if dt.now().second == 0:
    #     combined_df.to_csv("tick_stream.csv",sep="|")
    #     print(f"saved,{counter}",end='\r')
    # else:
    print(f"         {counter}",end='\r')


kite, kws = set_kite(kite_api_key="5ytarhiur9g1jebq",
    kite_access_token="oAwZpfJqmnb33FKt5wEczWCZmSdaTMKs")
instruments_df = get_instruments_book(kite)

next_expiry_datetime = instruments_df[\
                                    (instruments_df['name']=='NIFTY') 
                                    & (instruments_df['instrument_type'] == 'CE')]\
                                    ['expiry_datetime'].min()


# names = ['NIFTY']
names = ['NIFTY 50', 'INDIA VIX','NIFTY BANK', 'NIFTY','BANKNIFTY']
# names = ['NIFTY 50', 'INDIA VIX','NIFTY BANK']

tokens = list(instruments_df[(instruments_df['name']\
        .isin(names))&(instruments_df['expiry_datetime']==next_expiry_datetime)]\
            ['instrument_token'].astype(int))
print(type(tokens))
print(len(tokens))
# strike = 18_000.0
# underlying = 'NIFTY'
# expiry_datetime=dt(2022,1,27,15,30)
# fno_df = pd.DataFrame({'strike':[strike,strike,strike],\
#                         'underlying':[underlying,underlying,underlying],
#                         'expiry_datetime':[expiry_datetime,expiry_datetime,expiry_datetime],
#                         'call_put':['CE','PE','PE']})
# fno_df['tokens'] = get_multiple_fno_instrument_id(
#         instruments_df=instruments_df,
#         fno_df=fno_df)
# tokens = list(fno_df['tokens'])
# print(tokens)


<class 'list'>
464


In [2]:
# Assign the callbacks.
combined_df = pd.DataFrame()
kws.on_ticks = on_ticks
kws.on_connect = on_connect
kws.on_close = on_close
kws.on_error = on_error
kws.on_noreconnect = on_noreconnect
kws.on_reconnect = on_reconnect
kws.on_order_update = on_order_update

# kws.enable_reconnect(reconnect_interval=5, reconnect_tries=50)

# Infinite loop on the main thread. Nothing after this will run.
# You have to use the pre-defined callbacks to manage subscriptions.
# kws.connect(disable_ssl_verification=True)
kws.connect()

on connect: {"peer": "tcp4:13.127.118.127:443", "headers": {"date": "Mon, 24 Jan 2022 09:29:33 GMT", "connection": "upgrade", "upgrade": "websocket", "sec-websocket-accept": "h6FNntMirrG/aXf+PLjdol4KHBI="}, "version": 18, "protocol": null, "extensions": []}


Connection error: 1006 - connection was closed uncleanly (peer dropped the TCP connection without previous WebSocket closing handshake)
Connection closed: 1006 - connection was closed uncleanly (peer dropped the TCP connection without previous WebSocket closing handshake)


closed connection on error: 1006 connection was closed uncleanly (peer dropped the TCP connection without previous WebSocket closing handshake)
closed connection on close: 1006 connection was closed uncleanly (peer dropped the TCP connection without previous WebSocket closing handshake)


{'kotak_consumer_key': 'ug1QtNucZF4WLSinsfGfQ0sgjoYa',
 'kotak_access_token': 'd2c0a033-86d3-3b66-9b94-7241b0078fef',
 'kotak_consumer_secret': 'PGLsaajJLkpKLFXNO6pXrjCr_6Qa',
 'kotak_user_id': 'HG11011990',
 'kotak_access_code': '2825',
 'kotak_user_password': 'KSSh@r3d',
 'kite_api_key': '5ytarhiur9g1jebq',
 'kite_access_token': 'L4tDd3gw26Mq6QuJ6iMy6Xccy9KrgWWa',
 'kite_api_secret': 'q3hkmxt8b1gf4ftj5ew9x455sejb409h'}

'/Users/hg/Space/Code/AlgoTrader/broker_secret.json'